# Grid Search 실습

### 데이터셋 다운로드

In [24]:
! wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1GBtNoT3sC2ZjscrPKaFwb5pBEqLNrF9Z' -O pima-indians-diabetes.csv #https://drive.google.com/file/d/1GBtNoT3sC2ZjscrPKaFwb5pBEqLNrF9Z/view?usp=sharing

--2021-05-26 09:11:53--  https://docs.google.com/uc?export=download&id=1GBtNoT3sC2ZjscrPKaFwb5pBEqLNrF9Z
Resolving docs.google.com (docs.google.com)... 108.177.12.102, 108.177.12.100, 108.177.12.138, ...
Connecting to docs.google.com (docs.google.com)|108.177.12.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-1s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/bgur8al0mqdomt2l7sgltaljuh20qmge/1622020275000/04591858293269658552/*/1GBtNoT3sC2ZjscrPKaFwb5pBEqLNrF9Z?e=download [following]
--2021-05-26 09:11:53--  https://doc-0o-1s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/bgur8al0mqdomt2l7sgltaljuh20qmge/1622020275000/04591858293269658552/*/1GBtNoT3sC2ZjscrPKaFwb5pBEqLNrF9Z?e=download
Resolving doc-0o-1s-docs.googleusercontent.com (doc-0o-1s-docs.googleusercontent.com)... 172.217.204.132, 2607:f8b0:400c:c15::84
Connecting to doc-0o-1s-docs.googleusercontent.com (doc-0o-1s-

## batch size와 epoch 최적화

In [25]:
#@title
# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.688802 using {'batch_size': 20, 'epochs': 100}
0.619792 (0.024774) with: {'batch_size': 10, 'epochs': 10}
0.680990 (0.025976) with: {'batch_size': 10, 'epochs': 50}
0.671875 (0.069072) with: {'batch_size': 10, 'epochs': 100}
0.584635 (0.057084) with: {'batch_size': 20, 'epochs': 10}
0.667969 (0.009568) with: {'batch_size': 20, 'epochs': 50}
0.688802 (0.022402) with: {'batch_size': 20, 'epochs': 100}
0.583333 (0.044690) with: {'batch_size': 40, 'epochs': 10}
0.641927 (0.010253) with: {'batch_size': 40, 'epochs': 50}
0.665365 (0.012890) with: {'batch_size': 40, 'epochs': 100}
0.614583 (0.023939) with: {'batch_size': 60, 'epochs': 10}
0.602865 (0.062201) with: {'batch_size': 60, 'epochs': 50}
0.683594 (0.025315) with: {'batch_size': 60, 'epochs': 100}
0.518229 (0.042112) with: {'batch_size': 80, 'epochs': 10}
0.643229 (0.009744) with: {'batch_size': 80, 'epochs': 50}
0.639323 (0.032264) with: {'batch_size': 80, 'epochs': 100}
0.563802 (0.049445) with: {'batch_size': 100, 'epochs':

## 학습률과 모멘텀 최적화

In [26]:
#@title
# Use scikit-learn to grid search the learning rate and momentum
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
import time
# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, momentum=0):
	# create model
	modelx = Sequential()
	modelx.add(Dense(12, input_dim=8, activation='relu'))
	modelx.add(Dense(1, activation='sigmoid'))
	# Compile model
	optimizer = SGD(lr=learn_rate, momentum=momentum)
	modelx.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return modelx
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
modelx = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=modelx, param_grid=param_grid, n_jobs=-1, cv=3)

# 트레이닝 시작
start = time.time()
print("트레이닝 시작")
grid_result = grid.fit(X, Y)
end = time.time()
print(end - start, ' 초 결렸습니다')    

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

트레이닝 시작


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


475.28758120536804  초 결렸습니다
Best: 0.691406 using {'learn_rate': 0.001, 'momentum': 0.4}
0.641927 (0.046694) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.674479 (0.018136) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.691406 (0.012758) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.690104 (0.030978) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.690104 (0.038051) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.649740 (0.026557) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.677083 (0.004872) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.660156 (0.034499) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.648438 (0.028348) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.648438 (0.025315) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.647135 (0.021236) with: {'learn_rate': 0.01, 'momentum': 0.8}
0.651042 (0.024774) with: {'learn_rate': 0.01, 'momentum': 0.9}
0.651042 (0.024774) with: {'learn_rate': 0.1, 'momentum': 0.0}
0.651042 (0.024774) with: {'learn_rate': 0.1, 'momentum': 0.2}
0.651042 (0.

In [27]:
# Use scikit-learn to grid search the learning rate and momentum
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
import time
# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, momentum=0):
	# create model
	modelx = Sequential()
	modelx.add(Dense(12, input_dim=8, activation='relu'))
	modelx.add(Dense(1, activation='sigmoid'))
	# Compile model
	optimizer = SGD(lr=learn_rate, momentum=momentum)
	modelx.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return modelx
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
modelx = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
# learn_rate와 momentum 두 개의 하이퍼파라미터의 그리드를 코드로 정의하십시오. 
# learn_rate는 0.001, 0.01. 0.1, 0.2, 0.3로 하고 momentum은 0.0, 0.2, 0.4, 0.6, 0.8, 0.9로 하십시오.
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0,0.2,0.4,0.6,0.8,0.9]
param_grid = dict(learn_rate=learn_rate,momentum=momentum)
grid = GridSearchCV(estimator=modelx,param_grid=param_grid,n_jobs=-1)

# 트레이닝 시작
start = time.time()
print("트레이닝 시작")
grid_result = grid.fit(X, Y)
end = time.time()
print(end - start, ' 초 결렸습니다') 

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

트레이닝 시작


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


834.6626853942871  초 결렸습니다
Best: 0.690154 using {'learn_rate': 0.001, 'momentum': 0.2}
0.674561 (0.043063) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.690154 (0.037506) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.658951 (0.058341) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.670648 (0.061118) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.651108 (0.047215) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.671938 (0.039610) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.687599 (0.046354) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.653756 (0.051642) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.678465 (0.047002) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.653790 (0.055067) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.648561 (0.053109) with: {'learn_rate': 0.01, 'momentum': 0.8}
0.651159 (0.052445) with: {'learn_rate': 0.01, 'momentum': 0.9}
0.648561 (0.054210) with: {'learn_rate': 0.1, 'momentum': 0.0}
0.651159 (0.052445) with: {'learn_rate': 0.1, 'momentum': 0.2}
0.648561 (0.0

In [28]:

layer = tf.keras.layers.Dropout(rate=0.2, input_shape=(2,))

In [29]:
layer = tf.keras.layers.LeakyReLU(alpha=0.1)